# Using a Stewart Platform as a Motion Simulator
#### Python + ESP32 + 3D CAD + Motion Capture Data

This is a WIP progress!


[<img src="./doc/jupyter_nb/Stewart_IRL_1.gif" width="30%" height="30%">](./doc/jupyter_nb/Stewart_IRL_1.gif)
[<img src="./doc/jupyter_nb/Fusion360.png" width="30%" height="30%">](./doc/jupyter_nb/Fusion360.png)
[<img src="./doc/jupyter_nb/Stewart_IRL_2.gif" width="30%" height="30%">](./doc/jupyter_nb/Stewart_IRL_2.gif)


Import the CSV file containing the positions of a body in 6DOF. A 2DOF example of a person running on a treadmill is provided, of which we scale into the form of

Yaw: Radians

Translation in Z axis: Centimeters scaled according to height of motion simulator

In [ ]:
import csv
import numpy as np
import matplotlib.pyplot as plt

file = open('positions_processed.csv')
csvreader = csv.reader(file)
rows = []
for row in csvreader:
        rows.append(row)
rows
file.close()
movements = np.array(rows)

yaw_movements = movements[0,:].astype('float') /180*np.pi
z_movements = movements[1,:].astype('float')/20

plt.subplot(211) 
plt.plot(yaw_movements)
plt.subplot(212) 
plt.plot(z_movements)

My approach is then to
1. Use the stewart_py package to calculate the inverse kinematic of the desired position according to the movement data
2. Use py_serial to output the data to a ESP32
3. ESP32 receives serial data and issues I2C command to a pca9685
4. pca9685 commands 6 servo motors to rotate accordingly

This is a very cost efficient solution however if I could redo this project I'd use a Raspberry Pi instead and direcly use it to issue I2C commands to a pca9685

In [ ]:
import numpy as np
from src.stewart_controller import Stewart_Platform
import matplotlib.pyplot as plt
import time
import serial

# Initialize serial
ser = serial.Serial()
ser.port = 'COM5'
ser.baudrate = 38400
ser.setDTR(False)
ser.setRTS(False)

# ser.close()
if not ser.isOpen():
    ser.open()

# Call object, 
# These parameters are for my 3D models as provided, you can change your parameters according to your own hardware,
# Stewart_Platform(r_B, r_P, lhl, ldl, Psi_B, Psi_P), where
# r_B = Radius of Base (Bottom)
# r_P = Radius of Platform (Top)
# lhl = Servo Horn Length
# ldl = Rod length
# Psi_B = Half of angle between two anchors on the base
# Psi_P = Half of angle between two anchors on the platform
# For details please refer to my tutorial https://github.com/Yeok-c/Stewart_Py

platform = Stewart_Platform(13.2/2, 17.5/2, 5.08, 12, 0.2269, 0.82)
controller_freq = 12
# %matplotlib qt
# Initialize Plots
fig, ax = plt.subplots()    

while 1:
    # Loop through various angles
    for ix in range(0, 600, 8):
        time.sleep(1/controller_freq)
        # tic = time.time()

        # send servo angles for this time-step. Movement input data is scaled for better clarity, output is scaled into degrees
        servo_angles = platform.calculate( np.array([0,0,-2+0.5*z_movements[ix]]), np.array([0, 0, yaw_movements[ix]]) )/np.pi*180
        
        # Send serial data to ESP32 in the form of a string "<A,float,float,float,float,float,float>"
        towrite = np.array2string(servo_angles, precision=1, separator=',').strip('[]')
        towrite = '<A,' + towrite + '>'
        ser.write(towrite.encode())     # write a string
        print(towrite)

        # Hide this if your computer takes too long to plot, affecting the controller speed
        # ax = platform.plot_platform()
        # plt.pause(1/controller_freq)
        # plt.draw()

        # toc = time.time()
        # print(tic-toc) # Measure time to see if plotting affects speed of controller
